# Import libraries

In [1]:
%matplotlib inline

import torch
import torchvision
import imageio
import pandas as pd
import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from os import path

base_path = "D:\Vezdecode\CV\CV20"

# Detection car

In [2]:
def parser_result(result):
    pred = result.pred[0]
    names = result.names
    
    s = ""
    if pred.shape[0]:
        for c in pred[:, -1].unique():
            n = (pred[:, -1] == c).sum()  # Detections per class
            s += f"{names[int(c)]}, "  # Add to string
    else:
        s += "(no detections)"
        
    return s

def find_car(input_dir:str, output_cars:str="output.csv"):
    files = [path.join(input_dir, img) for img in os.listdir(input_dir)]
    
    result_df = pd.DataFrame(columns=["image", "is_car"])
    name_images = [] # A list for saving image names
    preds_is_car = [] # A list for saving the prediction of the presence of a car in the image
    
    model = torch.hub.load("ultralytics/yolov5", "yolov5s")
    
    for img in tqdm(files):
        name_images.append(img.split("\\")[-1])
        
        image = imageio.imread(img)
        result = model(image)
        pred = parser_result(result)
        
        if "car" in pred or "truck" in pred or "bus" in pred:
            preds_is_car.append(True)
        else:
            preds_is_car.append(False)
    
    result_df["image"] = name_images
    result_df["is_car"] = preds_is_car
            
    result_df.to_csv(output_cars, index=False, header=False)
    
    return result_df

In [3]:
input_dir = path.join(base_path, "resources\merged_data")

output_df = find_car(input_dir, "output.csv")

Using cache found in C:\Users\USER/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-6-18 Python-3.8.11 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce GTX 1050, 3072MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
valid_target = pd.read_csv(
    path.join(base_path, "resources\\val.csv"), 
    header=None, names=["image", "is_car"]
)

valid_target.head()

,image,is_car
0,00001.jpg,False
1,00002.jpg,False
2,00003.jpg,True
3,00004.jpg,False
4,00005.jpg,True


In [5]:
K = sum(output_df["is_car"] == valid_target["is_car"])
score = max (0, K - 60) * 20 / (100 - 60)
score

19.5